In [1]:
%load_ext autoreload

In [2]:
import pyxdf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from matplotlib.colors import ListedColormap
import pickle

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [3]:
from matplotlib import font_manager
import matplotlib as mpl

fontPath = '/usr/share/fonts/truetype/msttcorefonts/Times_New_Roman_Italic.ttf'
fontProp = font_manager.FontProperties(fname = fontPath)

font_manager.fontManager.addfont(fontPath)
mpl.rcParams['font.family'] = fontProp.get_name()

In [4]:
collateData = []
Age = []
Height = []
Weight = []
Elasticity = []
Hydration = []
Frequency = []

for subjectNumber in range(1, 100):
    if subjectNumber not in [5, 11, 14, 17, 18, 35, 38, 41]:
        name = "../formattedData/" + str(subjectNumber) + ".pkl"
        with open(name, "rb") as file:
            loadedData = pickle.load(file)

        Features = loadedData["EMG"]
        mean = np.mean(Features, axis = -1)
        std = np.std(Features, axis = -1)
        normalizedFeatures = (Features - mean[..., np.newaxis])/std[..., np.newaxis]
        Labels = loadedData["Labels"]
        samples = 2 * loadedData["Frequency"]

        Indices =  {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}
        for i in range(len(Labels)):
            Indices[Labels[i]].append(i)

        gesturesLabels = np.zeros((10, 36, 12, 12))
        for i in range(10):
            for j in range(36):
                gesturesLabels[i, j] = (1/samples) * normalizedFeatures[Indices[i][j], :, :] @ normalizedFeatures[Indices[i][j], :, :].T
        
        collateData.append(gesturesLabels)
        Age.append(loadedData["Physiology"]["Age"])
        Height.append(loadedData["Physiology"]["Height"])
        Weight.append(loadedData["Physiology"]["Weight"])
        Elasticity.append(loadedData["Physiology"]["Skin_Elasticity"])
        Hydration.append(loadedData["Physiology"]["Skin_Hydration"])
        Frequency.append(loadedData["Frequency"])

In [5]:
collateData = np.array(collateData)
Age = np.array(Age)
Height = np.array(Height)
Weight = np.array(Weight)
Hydration = np.array(Hydration)
Elasticity = np.array(Elasticity)
Frequency = np.array(Frequency)

print(collateData.shape)
print(Age.shape)
print(Weight.shape)
print(Height.shape)
print(Frequency.shape)
print(Hydration.shape)
print(Elasticity.shape)

(91, 10, 36, 12, 12)
(91,)
(91,)
(91,)
(91,)
(91,)
(91,)


In [6]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [7]:
dev = "cpu" 
device = torch.device(dev)

In [8]:
from spdLearning import spdNN
from spdLearning import optimizers 
from spdLearning import trainTest
from spdLearning import spdNet

In [11]:
allAccuracy = []
for subject in range(91):
    trainFeatures = np.zeros((180, 12, 12))
    trainLabels = np.zeros((180))
    count = 0
    for i in range(10):
        trainFeatures[count:count + 18] = collateData[subject, i, :18]
        trainLabels[count:count + 18] = [i] * 18
        count += 18

    testFeatures = np.zeros((180, 12, 12))
    testLabels = np.zeros((180))
    count = 0
    for i in range(10):
        testFeatures[count:count + 18] = collateData[subject, i, 18:]
        testLabels[count:count + 18] = [i] * 18
        count += 18

    trainDataset = BaseDataset(trainFeatures, trainLabels)
    testDataset = BaseDataset(testFeatures, testLabels)
    trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
    testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

    numberEpochs = 1000

    model = spdNet.learnSPDMatrices(10).to(device)
    numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(numParams)

    lossFunction = nn.CrossEntropyLoss()
    spdOptimizer = optimizers.MixOptimizer(model.parameters(), lr = 0.05)

    maxValue = 0
    for epoch in range(numberEpochs):
        trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, spdOptimizer, lossFunction)
        testLoss, testAccuracy = trainTest.testOperation(model, device, testDataloader, lossFunction)
        if maxValue < testAccuracy:
            maxValue = testAccuracy
            torch.save(model.state_dict(), str(subject) + '.pt')
    print(maxValue)
    allAccuracy.append(maxValue)
    print(" ")

1496
75.55555555555556
 
1496
86.66666666666667
 
1496
66.11111111111111
 
1496
95.0
 
1496
62.22222222222222
 
1496
87.77777777777777
 
1496
26.666666666666668
 
1496
71.11111111111111
 
1496
57.22222222222222
 
1496
72.22222222222223
 
1496
70.55555555555556
 
1496
86.11111111111111
 
1496
36.111111111111114
 
1496
94.44444444444444
 
1496
78.88888888888889
 
1496
92.22222222222223
 
1496
72.22222222222223
 
1496
76.11111111111111
 
1496
86.11111111111111
 
1496
48.333333333333336
 
1496
88.88888888888889
 
1496
96.11111111111111
 
1496
98.88888888888889
 
1496
98.33333333333333
 
1496
98.88888888888889
 
1496
72.22222222222223
 
1496
61.111111111111114
 
1496
73.33333333333333
 
1496
93.33333333333333
 
1496
99.44444444444444
 
1496
88.88888888888889
 
1496
96.11111111111111
 
1496
91.66666666666667
 
1496
73.88888888888889
 
1496
67.22222222222223
 
1496
46.666666666666664
 
1496
87.77777777777777
 
1496
91.66666666666667
 
1496
97.77777777777777
 
1496
60.55555555555556
 
1496
79.

In [12]:
print(np.mean(allAccuracy))

81.33089133089135


In [13]:
np.save("SPDSplitA.npy", allAccuracy)

In [ ]:
%autoreload